In [1]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json

In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4],
  autocommit=True
)

mycursor = mydb.cursor(dictionary=True)

In [3]:
def exec_to_result(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def get_stock_result(code):
    return exec_to_result("select stok_fisik,right(no_so,3) depo, namaDepo \
from inventori_so_detail inner join masterf_depo on right(no_so,3) = masterf_depo.id \
where no_so like 'so2112%' and kode_obat = '"+code+"' and stok_fisik > 0 order by stok_fisik desc")
    
def get_mutasi(kode,sel):
  h = get_stock_result(kode)
  T = 'P02' if sel < 0 else 'P01'
  t = abs(sel)
  if(sel < 0):
    for i in h:
      if (t >= i['stok_fisik']):
          t -= i['stok_fisik']
          i['target'] = 0
          i['trans'] = T
      else:
          i['target'] = i['stok_fisik'] - t
          t = 0
          i['trans'] = T
  else:
    return
  D = []
  for i in h:
    if i['stok_fisik'] != i['target']:
      D.append({'depo':i['depo'],'old_stok':i['stok_fisik'],'new_stok':i['target'],'mutasi':abs(i['target'] - i['stok_fisik']),'trans':T,\
                'kode':'|'+kode+'|','namaDepo':i['namaDepo']})
  return D

In [17]:
import decimal
R = []
for i in range(2334,2334):
  N = get_mutasi(kList[i],decimal.Decimal(sList[i]))
  if N:
   R = R + N
RDF = pd.DataFrame(R)
RDF

""


In [15]:
RDF.to_csv("RDF4.csv")

In [19]:
for i in range(0,2333):
  N = get_mutasi(kList[i],decimal.Decimal(sList[i]))
  if not N:
      print(kList[i])


020700653
